In [14]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
%pip install langchain_community langchain_google_genai langgraph langchain_text_splitters 

  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl (435 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install beautifulsoup4 faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [ ]:
# 검색해서 뉴스를 가져오는 코드 


# 1. 데이터를 가져오고 
from langchain.document_loaders import WebBaseLoader
import bs4

# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

# 2. 데이터를 청킹하고 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

# 3. 데이터를 저장해주어야함
vectorstore = FAISS.from_documents(documents=texts, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriever=vectorstore.as_retriever()

# 4. prompt 간단하게 설정
from langchain_core.prompts import PromptTemplate

prompt=PromptTemplate.from_template("""
당신은 훌륭한 AI 어시스턴스 입니다.
아래의 컨텍스트를 참고해서 답변을 잘 주세요.

{context}

{question}
"""

# 5. langchain만들기 

llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", temperature=0)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("이 기사의 주제는 무엇인가요?")



문서의 수: 1


TypeError: unsupported operand type(s) for |: 'dict' and 'str'